In [1]:
import numpy as np
import pandas as pd
import cv2 as cv
import bbox_visualizer as bbv
import matplotlib.pyplot as plt
from PIL import __version__ as PILLOW_VERSION
from torchvision import datasets
import torch
from torch.utils.data import Dataset
from PIL import Image
import os
import bbox_visualizer as bbv
import shutil

In [ ]:
DATA_ROOT = "data/raw/HW1_dataset/"

with open(os.path.join(DATA_ROOT, "classes.names"), "r") as f:
    classes_encoding = {name.strip("\n"): i for i, name in enumerate(f.readlines())}
print(classes_encoding)
# sanity check for missing labels
bbox_labels = {name.split(".")[0] for name in os.listdir(os.path.join(DATA_ROOT, "bboxes_labels"))}
_images = {name.split(".")[0] for name in os.listdir(os.path.join(DATA_ROOT, "images"))}
missing_labels = bbox_labels.symmetric_difference(_images)
print(missing_labels)

In [ ]:
# remove files without labels
image_filenames = [filename for filename in os.listdir(os.path.join(DATA_ROOT, "images")) if filename.split(".")[0] not in missing_labels]
label_filenames = os.listdir(os.path.join(DATA_ROOT, "bboxes_labels"))

# sanity check that order is maintained so that labels still match files
for label_filename, image_filename in zip(label_filenames, image_filenames):
    label_filename, image_filename = label_filename.split(".")[0], image_filename.split(".")[0]
    if label_filename != image_filename:
        print(label_filename, image_filename)
        break

print(len(image_filenames))

In [ ]:
def reverse_transform_format(label, w_img, h_img):
    cls, x_center, y_center, w, h, = label
    w_box = w * w_img
    h_box = h * h_img
    x_min = x_center * w_img - w_box / 2
    x_max = x_center * w_img + w_box / 2
    y_min = y_center * h_img - h_box / 2
    y_max = y_center * h_img + h_box / 2
    return [cls, x_min, y_min, x_max, y_max]


class ImageBBoxDataset(Dataset):
    """ This assumes that all images are of the same size """

    def __init__(self, root=DATA_ROOT, label_folder="bboxes_labels", image_folder="images", labels=label_filenames, images=image_filenames,
                 transform=None, invert_darknet=False):
        if len(images) != len(labels):
            raise RuntimeError("labels and images count dont match")
        self.root = root
        self.label_folder = label_folder
        self.image_folder = image_folder
        self.label_filenames = labels
        self.images = images
        self.labels = []
        self.transform = transform
        self.invert = invert_darknet

        temp_img = Image.open(os.path.join(self.root, self.image_folder, self.images[0]))
        self.w_img, self.h_img = temp_img.size

        for label_filename in self.label_filenames:
            with open(os.path.join(self.root, self.label_folder, label_filename), "r") as f:
                lines = f.readlines()
                if self.invert:
                    bbox_label = [reverse_transform_format([int(cls), float(x), float(y), float(w), float(h)], self.w_img, self.h_img)
                                  for cls, x, y, w, h in [line.split(" ") for line in lines]]
                else:
                    bbox_label = [[int(cls), float(x), float(y), float(w), float(h)] for cls, x, y, w, h in [line.split(" ") for line in lines]]
                self.labels.append(bbox_label)

    def __getitem__(self, index):
        image_path = os.path.join(self.root, self.image_folder, self.images[index])
        x = cv.imread(image_path)[..., ::-1]
        y = self.labels[index]
        if self.transform is not None:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.images)

    def get_paths(self, index):
        image_path = os.path.join(self.root, self.image_folder, self.images[index])
        label_path = os.path.join(self.root, self.label_folder, self.label_filenames[index])
        return image_path, label_path

ds = ImageBBoxDataset()

In [ ]:
number_class = {v: k for k, v in classes_encoding.items()}
class_color = {0: (255, 0, 0), 1: (0, 255, 0), 2: (0, 0, 255), 3: (255, 255, 0), 4: (0, 255, 255), 5: (255, 0, 255), 6: (128, 128, 128), 7: (128, 0, 128)}
frame, full_label = ds[0]

def draw_bboxes_and_classes(frame_img, label):
    bboxes = [[round(coord) for coord in box[1:]] for box in label]
    classes = [number_class[lbl[0]] for lbl in label]

    for box, cls in zip(bboxes, classes):
        color = class_color[classes_encoding[cls]]
        frame_img = bbv.draw_multiple_rectangles(frame_img, [box], bbox_color=color)
        frame_img = bbv.add_multiple_labels(frame_img, [cls], [box], text_bg_color=color)

    cv.imshow('Frame', frame_img)
    cv.waitKey(0)
    cv.destroyAllWindows()
    cv.waitKey(1)

In [7]:
!python yolov5_ws/yolov5/detect.py --weights yolov5_ws/yolov5/runs/train/exp5/weights/best.pt --img 640 --conf 0.4 --source data/raw/images/test --save-txt

detect: weights=['yolov5_ws/yolov5/runs/train/exp5/weights/best.pt'], source=data/raw/images/test, data=yolov5_ws\yolov5\data\coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5_ws\yolov5\runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  2022-12-24 Python-3.9.13 torch-1.13.0 CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7031701 parameters, 0 gradients
image 1/200 C:\projects\cvor1\data\raw\images\test\P016_tissue1_1329.jpg: 480x640 1 Right_Needle_driver, 1 Left_Empty, 174.8ms
image 2/200 C:\projects\cvor1\data\raw\images\test\P016_tissue1_1348.jpg: 480x640 1 Left_Empty, 181.8ms
image 3/200 C:\projects\cvor1\data\raw\images\test\P016_tissue1_144.jpg: 480x640 1 Right_Needle_driver, 1 Left_Em